## Particle Swarm Optimization - Nearest Neighbor Velocity Matching

### Modelagem da particula:

In [ ]:
import numpy as np

# Modelando as particulas
class Particle:
  def __init__(self, num_dimencoes, bounds):
    '''
    Construtor
    '''
    self.num_dimencoes = num_dimencoes
    self.posicao = []
    self.velocidade = []
    self.nova_velocidade = self.velocidade
    self.closest = None
    self.min_dist = np.inf
    self.bounds = bounds

    # Definir valores aleatorios de velocidade e posicao.
    # Para cada dimensao, por exemplo x e y, informada estabelece um valor aleatorio para cada particula
    for i in range(self.num_dimencoes):
      self.posicao.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1])) # estabelece limite de valores para cada particula
      self.velocidade.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))

  @staticmethod #################
  def _euclidean_distance(p1, p2):
    '''
    Calcula a distância euclidiana entre a particula 1 e 2 
    '''
    # extrai as posicoes de cada particula
    posicao_1 = np.array(p1.posicao)
    posicao_2 = np.array(p2.posicao)
    # Calcula a distância euclidiana entre a particula 1 e 2 
    distance = np.sqrt(sum((posicao_1 - posicao_2)**2))

    return distance

  def find_closest(self, swarm):
    '''
    Para cada partícula no enxame calcular a partícula mais próxima
    '''
    # pra cada particula do enxame, ele ira encontrar o mais proximo
    #se a particula for igual a ela mesma na analise, ela continuara a busca
    for particle in swarm:
      if particle == self:
        continue
      dist = Particle._euclidean_distance(self, particle)
      #verifica se a distancia eh menor que a minima distancia, se for a distancia minima sera igual a dist
      if dist < self.min_dist:
        self.min_dist = dist
        self.closest = particle

  def adjust_velocity(self):
    '''
    Ajustar a nova velocidade baseado na velocidade do vizinho
    '''
    self.nova_velocidade = self.closest.velocidade

  def update_velocity(self):
    '''
    Atualizar a nova velocidade
    '''
    self.velocidade = self.nova_velocidade

  def apply_craziness(self):
    '''
    Aplicar a operação 'craziness' no indivíduo
    '''
    # o craziness representa a aleatoriedade
    craziness_velocity = []
    # percorre as dimensoes que temos, e adiciona no craziness_velocity um valor aleatorio
    for i in range(self.num_dimencoes):
      craziness_velocity.append(np.random.uniform(
        self.bounds[i][0], self.bounds[i][1]))
    self.velocidade = craziness_velocity

  def update_position(self):
    '''
    Atualizando a posição
    '''
    # Atualizando a posicao baseada na velocidade
    self.posicao = list(np.array(self.posicao) + np.array(self.velocidade))


### Modelagem do enxame:

In [ ]:
class Swarm:

  def __init__(self, particles):
    '''
    Construtor
    '''
    # recebe uma lista de particulas criadas
    self.particles = particles

  def swarm_closest(self):
    '''
    Percorre todas as partículas e chama as funções para encontrar a partícula mais próxima e ajustar a nova velocidade
    '''
    # encontra a particula mais proxima de todo enxame e ja ajusta a velocidade
    for p in self.particles:
      p.find_closest(self.particles)
      p.adjust_velocity()

  def swarm_update_velocities(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a velocidade
    '''
    for p in self.particles:
      p.update_velocity()

  def swarm_craziness(self, craziness_threshold):
    '''
    Percorre todas as partículas e chama a função para aplicar "craziness"
    '''
    # percorre as particulas, pega um valor aleatorio entre 0 e 1 e testa no valor aleatorio(treshold), se for < aplica o craziness
    for p in self.particles:
      if np.random.uniform(0, 1) < craziness_threshold:
        p.apply_craziness()

  def swarm_update_positions(self):
    '''
    Percorre todas as partículas e chama a função para atualizar a posição
    '''
    for p in self.particles:
      p.update_position()

### Código para plotagem do PSO:

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import glob
import os
import shutil

class PlotUtils:

  directory = "pso_plots"
  filename = 'pso.gif'

  @staticmethod
  def start_plot():
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)
    if not os.path.exists(PlotUtils.directory):
      os.makedirs(PlotUtils.directory)

  @staticmethod
  def plot_particle(particle):
    plt.scatter(particle.posicao[0], particle.posicao[1])

  @staticmethod
  def plot_iteration(i):
    plt.title(f"PSO {i}")
    plt.xlim(-1500, 1500)
    plt.ylim(-1500, 1500)
    plt.xlabel('x[0]')
    plt.ylabel('x[1]')
    iteration = str(i).zfill(5)
    plt.savefig(
      f"{PlotUtils.directory}/iteration_{iteration}.png", facecolor="white", dpi=75)
    plt.close()

  @staticmethod
  def save():
    images = [Image.open(f) for f in sorted(
      glob.glob(PlotUtils.directory+"/*"))]
    img = images[0]
    img.save(fp=PlotUtils.filename, format='GIF',
              append_images=images, save_all=True, duration=200, loop=0)
    if os.path.exists(PlotUtils.directory):
      shutil.rmtree(PlotUtils.directory)

### Execução:

In [ ]:
 def executar_pso(num_particulas, num_iteracoes,
                  num_dimencoes, limites, prob_craziness):
  PlotUtils.start_plot()
  print("inicialização")

  # cria uma lista, e vai salvando os valores das dimensoes 
  particles = []
  for i in range(num_particulas):
    particles.append(Particle(num_dimencoes, limites))

  swarm = Swarm(particles)

  print("começando as iterações")
  i = 0
  while i < num_iteracoes:
    print(f"iteração {i}")

    #Pode faltar algum swarm
    swarm.swarm_closest()
    swarm.swarm_update_velocities()
    swarm.swarm_craziness(prob_craziness)
    swarm.swarm_update_positions()

    for p in swarm.particles:
      PlotUtils.plot_particle(p)
    PlotUtils.plot_iteration(i)

    i += 1

  PlotUtils.save()

executar_pso(
  prob_craziness = 0.02,
  num_iteracoes = 25,
  num_particulas = 120,
  num_dimencoes = 2,
  limites = [(-50, 50), (-50, 50)])


inicialização
começando as iterações
iteração 0
iteração 1
iteração 2
iteração 3
iteração 4
iteração 5
iteração 6
iteração 7
iteração 8
iteração 9
iteração 10
iteração 11
iteração 12
iteração 13
iteração 14
iteração 15
iteração 16
iteração 17
iteração 18
iteração 19
iteração 20
iteração 21
iteração 22
iteração 23
iteração 24
